In [37]:
from openbb import obb
import warnings
import time
import pandas as pd
import yfinance as yf
from datetime import datetime, timedelta

# Ignore all warnings
warnings.filterwarnings("ignore")

def fetch_option_data(symbols, annualized_return_threshold, dte_threshold):
    all_results = []
    for symbol in symbols:
        # Get the option chains for the specified symbol
        chains = obb.derivatives.options.chains(symbol=symbol)
        df = chains.to_dataframe()

        # Filter for put options
        put_options = df[df['option_type'] == 'put']

        # Get the current underlying price
        current_price = df['underlying_price'].iloc[0]

        # Function to calculate annualized return
        def calculate_annualized_return(row):
            days_to_expiration = row['dte']
            strike_price = row['strike']
            bid_price = row['bid']
            
            if days_to_expiration == 0 or strike_price == 0:
                return 0
            
            # Calculate potential profit
            potential_profit = bid_price
            
            # Calculate annualized return
            annualized_return = (potential_profit / strike_price) * (365 / days_to_expiration) * 100
            
            return annualized_return

        # Apply the function to each row
        put_options['annualized_return'] = put_options.apply(calculate_annualized_return, axis=1)

        # Filter for options with annualized return higher than the threshold and strikes under the current price
        high_return_puts = put_options[
            (put_options['annualized_return'] >= annualized_return_threshold) & 
            (put_options['strike'] < current_price)
        ]

        if dte_threshold:
            # Filter for options with specific DTE
            high_return_puts = high_return_puts[high_return_puts['dte'] == dte_threshold]

        # Calculate percentage difference
        high_return_puts['pct_difference'] = (
            high_return_puts['strike'] - high_return_puts['underlying_price']
        ) / high_return_puts['underlying_price']

        # Add symbol column
        high_return_puts['symbol'] = symbol

        # Reset index
        high_return_puts.reset_index(drop=True, inplace=True)

        # Append to all_results
        all_results.append(high_return_puts)

        time.sleep(1)  # To respect API rate limits

    # Combine all results into a single DataFrame
    combined_results = pd.concat(all_results, ignore_index=True)
    return combined_results

def calculate_technical_indicators(symbols):
    technicals = {}
    for symbol in symbols:
        end_date = datetime.now()
        start_date = end_date - timedelta(days=365)  # Get 1 year of data
        
        stock = yf.Ticker(symbol)
        history = stock.history(start=start_date, end=end_date)
        
        # Calculate 200-day SMA
        sma_200 = history['Close'].rolling(window=200).mean().iloc[-1]
        
        # Calculate weekly RSI
        weekly_data = history.resample('W').last()
        delta = weekly_data['Close'].diff()
        gain = (delta.where(delta > 0, 0)).rolling(window=14).mean()
        loss = (-delta.where(delta < 0, 0)).rolling(window=14).mean()
        rs = gain / loss
        rsi = 100 - (100 / (1 + rs)).iloc[-1]
        
        # Calculate weekly MACD
        exp1 = weekly_data['Close'].ewm(span=12, adjust=False).mean()
        exp2 = weekly_data['Close'].ewm(span=26, adjust=False).mean()
        macd = exp1 - exp2
        signal = macd.ewm(span=9, adjust=False).mean()
        histogram = macd - signal
        
        technicals[symbol] = {
            'sma_200': sma_200,
            'weekly_rsi': rsi,
            'weekly_macd': macd.iloc[-1],
            'weekly_macd_signal': signal.iloc[-1],
            'weekly_macd_histogram': histogram.iloc[-1]
        }
    return technicals

In [38]:
symbols = ['AEO', 'HAL', 'HPQ', 'NVDL', 'PINS', 'PLTR', 'SOFI', 'STNE', 'TSLL', 'UAA', 'UAL', 'URBN']
annualized_return_threshold = 30
dte_threshold = 3

# Fetch option data
combined_results = fetch_option_data(symbols, annualized_return_threshold, dte_threshold)

# Calculate technical indicators
technicals = calculate_technical_indicators(symbols)

# Merge technical indicators into the combined_results DataFrame
combined_results['weekly_rsi'] = combined_results['symbol'].apply(lambda x: technicals[x]['weekly_rsi'])
combined_results['weekly_macd'] = combined_results['symbol'].apply(lambda x: technicals[x]['weekly_macd'])
combined_results['weekly_macd_signal'] = combined_results['symbol'].apply(lambda x: technicals[x]['weekly_macd_signal'])
combined_results['weekly_macd_histogram'] = combined_results['symbol'].apply(lambda x: technicals[x]['weekly_macd_histogram'])
combined_results['sma_200'] = combined_results['symbol'].apply(lambda x: technicals[x]['sma_200'])
combined_results['price_to_sma_200'] = (
    (combined_results['underlying_price'] - combined_results['sma_200']) / combined_results['sma_200']
) * 100

In [39]:
# Example: Filter options where weekly RSI is below 30 and sort by annualized return
filtered_results = combined_results[combined_results['weekly_rsi'] < 30]
sorted_results = filtered_results.sort_values(by='annualized_return', ascending=False)

# Display the top results
print(sorted_results.head())

   underlying_symbol  underlying_price      contract_symbol  expiration  dte  \
27              PINS             32.66  PINS241004P00032500  2024-10-04    3   
26              PINS             32.66  PINS241004P00032000  2024-10-04    3   
24              PINS             32.66  PINS241004P00031000  2024-10-04    3   
25              PINS             32.66  PINS241004P00031500  2024-10-04    3   

    strike option_type  open_interest  volume  last_trade_price  ... currency  \
27    32.5         put            455     366              0.47  ...      USD   
26    32.0         put            217     624              0.26  ...      USD   
24    31.0         put            150     406              0.11  ...      USD   
25    31.5         put            275     515              0.13  ...      USD   

    annualized_return  pct_difference  symbol  weekly_rsi  weekly_macd  \
27         160.974359       -0.004899    PINS    27.82423    -1.735583   
26          98.854167       -0.020208    PINS

In [40]:
from IPython.display import display
from ipywidgets import interact

def interactive_filter(rsi_threshold=30, min_annual_return=30, min_pct_difference=-0.15, max_pct_difference=-0.03, max_price_to_sma_200=25):
    filtered = combined_results[
        (combined_results['weekly_rsi'] >= rsi_threshold) &
        (combined_results['annualized_return'] >= min_annual_return) &
        (combined_results['pct_difference'] >= min_pct_difference) &
        (combined_results['pct_difference'] <= max_pct_difference) &
        (combined_results['price_to_sma_200'] <= max_price_to_sma_200)
    ]
    columns_to_display = ['underlying_symbol', 'underlying_price', 'strike', 'open_interest', 'bid', 'annualized_return', 'weekly_rsi', 'price_to_sma_200']
    display(filtered[columns_to_display].sort_values(by='annualized_return', ascending=False))

interact(
    interactive_filter,
    rsi_threshold=(30, 100, 1),
    min_annual_return=(30, 100, 1),
    min_pct_difference=(-0.25, 0, 0.001),
    max_pct_difference=(-0.25, 0, 0.001),
    max_price_to_sma_200=(0, 100, 1)
)

interactive(children=(IntSlider(value=30, description='rsi_threshold', min=30), IntSlider(value=30, descriptio…

<function __main__.interactive_filter(rsi_threshold=30, min_annual_return=30, min_pct_difference=-0.15, max_pct_difference=-0.03, max_price_to_sma_200=25)>

In [36]:
get_csp_options('NVDL', 30)

NameError: name 'get_csp_options' is not defined

In [ ]:
from datetime import datetime, timedelta

sma_period = 200
rsi_period = 14

# Calculate the start date
end_date = datetime.now()
start_date = (end_date - timedelta(days=max(sma_period, rsi_period * 7))).strftime('%Y-%m-%d')

# Fetch stock data for NVDL
stock_data = obb.equity.price.historical("NVDL", start_date=start_date, interval="1d")

stock_data_df = stock_data.to_dataframe()

# Calculate 200-day Simple Moving Average (SMA)
sma_200 = obb.technical.sma(stock_data_df['Close'], sma_period).iloc[-1]

# Calculate weekly RSI
weekly_data = stock_data.resample('W').last()
weekly_rsi = obb.technical.rsi(weekly_data['Close'], rsi_period).iloc[-1]

# Calculate weekly MACD
weekly_macd = obb.technical.macd(weekly_data['Close'])
macd_line = weekly_macd['MACD'].iloc[-1]
signal_line = weekly_macd['Signal'].iloc[-1]
macd_histogram = weekly_macd['Histogram'].iloc[-1]

print(f"200-day SMA: {sma_200:.2f}")
print(f"Weekly RSI: {weekly_rsi:.2f}")
print(f"Weekly MACD:")
print(f"  MACD Line: {macd_line:.2f}")
print(f"  Signal Line: {signal_line:.2f}")
print(f"  Histogram: {macd_histogram:.2f}")


AttributeError: 'App' object has no attribute 'technical'